In [13]:
from typing import Tuple
import hydra
from hydra.utils import instantiate
import logging
from omegaconf import DictConfig
from pathlib import Path
import pytorch_lightning as pl
from torch.utils.data import DataLoader

from navsim.planning.training.dataset import CacheOnlyDataset, Dataset
from navsim.planning.training.agent_lightning_module import AgentLightningModule
from navsim.common.dataloader import SceneLoader
from navsim.common.dataclasses import SceneFilter
from navsim.agents.abstract_agent import AbstractAgent

/root/miniconda3/envs/navsim/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:

def build_datasets(cfg: DictConfig, agent: AbstractAgent) -> Tuple[Dataset, Dataset]:
    train_scene_filter: SceneFilter = instantiate(cfg.scene_filter)
    if train_scene_filter.log_names is not None:
        train_scene_filter.log_names = [l for l in train_scene_filter.log_names if l in cfg.train_logs]
    else:
        train_scene_filter.log_names = cfg.train_logs

    val_scene_filter: SceneFilter = instantiate(cfg.scene_filter)
    if val_scene_filter.log_names is not None:
        val_scene_filter.log_names = [l for l in val_scene_filter.log_names if l in cfg.val_logs]
    else:
        val_scene_filter.log_names = cfg.val_logs

    data_path = Path(cfg.navsim_log_path)
    sensor_blobs_path = Path(cfg.sensor_blobs_path)

    train_scene_loader = SceneLoader(
        sensor_blobs_path=sensor_blobs_path,
        data_path=data_path,
        scene_filter=train_scene_filter,
        sensor_config=agent.get_sensor_config(),
    )

    val_scene_loader = SceneLoader(
        sensor_blobs_path=sensor_blobs_path,
        data_path=data_path,
        scene_filter=val_scene_filter,
        sensor_config=agent.get_sensor_config(),
    )

    train_data = Dataset(
        scene_loader=train_scene_loader,
        feature_builders=agent.get_feature_builders(),
        target_builders=agent.get_target_builders(),
        cache_path=cfg.cache_path,
        force_cache_computation=cfg.force_cache_computation,
    )

    val_data = Dataset(
        scene_loader=val_scene_loader,
        feature_builders=agent.get_feature_builders(),
        target_builders=agent.get_target_builders(),
        cache_path=cfg.cache_path,
        force_cache_computation=cfg.force_cache_computation,
    )

    return train_data, val_data

In [15]:
from omegaconf import OmegaConf
cfg = OmegaConf.load("/root/navsim/cfg.yaml")

In [16]:
pl.seed_everything(0, workers=True)

Seed set to 0


0

In [17]:
agent: AbstractAgent = instantiate(cfg.agent)

In [18]:
lightning_module = AgentLightningModule(
        agent=agent,
    )

In [19]:
train_data, val_data = build_datasets(cfg, agent)

Loading logs: 100%|██████████| 10/10 [00:00<00:00, 21.43it/s]
now let's begin to cache again!
Caching Dataset: 0it [00:00, ?it/s]
now let's begin to cache again!
Caching Dataset: 0it [00:00, ?it/s]


In [20]:
train_dataloader = DataLoader(train_data, **cfg.dataloader.params, shuffle=True)

In [21]:
val_dataloader = DataLoader(val_data, **cfg.dataloader.params, shuffle=False)

In [22]:
data=train_data[0]

In [23]:
train_data[0]

({'camera_feature': tensor([[[[0.3529, 0.3373, 0.3412,  ..., 0.1843, 0.2431, 0.2000],
            [0.3373, 0.3373, 0.3294,  ..., 0.1765, 0.2392, 0.2078],
            [0.3373, 0.3412, 0.3216,  ..., 0.1804, 0.2392, 0.2157],
            ...,
            [0.2235, 0.2275, 0.2196,  ..., 0.1843, 0.1804, 0.1804],
            [0.2157, 0.2275, 0.2196,  ..., 0.1882, 0.1843, 0.1804],
            [0.2275, 0.2235, 0.2314,  ..., 0.1804, 0.1804, 0.1765]],
  
           [[0.3490, 0.3333, 0.3373,  ..., 0.1882, 0.2431, 0.1961],
            [0.3333, 0.3333, 0.3137,  ..., 0.1961, 0.2392, 0.2039],
            [0.3333, 0.3373, 0.3059,  ..., 0.2000, 0.2392, 0.2118],
            ...,
            [0.2431, 0.2471, 0.2392,  ..., 0.2196, 0.2078, 0.2078],
            [0.2353, 0.2471, 0.2392,  ..., 0.2235, 0.2118, 0.2078],
            [0.2471, 0.2431, 0.2510,  ..., 0.2157, 0.2078, 0.2039]],
  
           [[0.3333, 0.3176, 0.3216,  ..., 0.2078, 0.2510, 0.2196],
            [0.3176, 0.3176, 0.3020,  ..., 0.2196, 0.247

In [24]:
data1=train_data[1]

In [25]:
data

({'camera_feature': tensor([[[[0.3529, 0.3373, 0.3412,  ..., 0.1843, 0.2431, 0.2000],
            [0.3373, 0.3373, 0.3294,  ..., 0.1765, 0.2392, 0.2078],
            [0.3373, 0.3412, 0.3216,  ..., 0.1804, 0.2392, 0.2157],
            ...,
            [0.2235, 0.2275, 0.2196,  ..., 0.1843, 0.1804, 0.1804],
            [0.2157, 0.2275, 0.2196,  ..., 0.1882, 0.1843, 0.1804],
            [0.2275, 0.2235, 0.2314,  ..., 0.1804, 0.1804, 0.1765]],
  
           [[0.3490, 0.3333, 0.3373,  ..., 0.1882, 0.2431, 0.1961],
            [0.3333, 0.3333, 0.3137,  ..., 0.1961, 0.2392, 0.2039],
            [0.3333, 0.3373, 0.3059,  ..., 0.2000, 0.2392, 0.2118],
            ...,
            [0.2431, 0.2471, 0.2392,  ..., 0.2196, 0.2078, 0.2078],
            [0.2353, 0.2471, 0.2392,  ..., 0.2235, 0.2118, 0.2078],
            [0.2471, 0.2431, 0.2510,  ..., 0.2157, 0.2078, 0.2039]],
  
           [[0.3333, 0.3176, 0.3216,  ..., 0.2078, 0.2510, 0.2196],
            [0.3176, 0.3176, 0.3020,  ..., 0.2196, 0.247

In [27]:
data1[0]["past_trajectory"]

tensor([[-1.0344e+01, -7.0251e+00, -3.5854e+00,  0.0000e+00],
        [-1.2816e-02, -2.3183e-02, -6.5973e-03,  0.0000e+00],
        [ 6.2080e-03,  3.4136e-03,  1.0786e-03,  0.0000e+00]])

In [ ]:
train_data[5][1]["trajectory"]

tensor([[ 4.2192e+00,  2.7789e-02,  1.9091e-03],
        [ 8.4631e+00,  4.8937e-02,  1.7613e-03],
        [ 1.2765e+01,  4.6216e-02,  1.8201e-04],
        [ 1.7097e+01,  2.5612e-02, -4.0979e-03],
        [ 2.1454e+01,  1.3250e-04, -7.1817e-03],
        [ 2.5819e+01, -3.8013e-02, -9.4086e-03],
        [ 3.0199e+01, -1.1229e-01, -1.2409e-02],
        [ 3.4576e+01, -1.9047e-01, -1.9861e-02]], dtype=torch.float64)

In [ ]:
data[0]["camera_feature"].shape

torch.Size([4, 3, 256, 1024])

In [ ]:
data[0]["lidar_feature"].shape

torch.Size([4, 1, 256, 256])

In [ ]:
data[0]["status_feature"].shape

torch.Size([8])

In [ ]:
data[1]["agent_states"].shape

torch.Size([30, 5])

In [ ]:

train_dataloader = DataLoader(train_data, **cfg.dataloader.params, shuffle=True)

val_dataloader = DataLoader(val_data, **cfg.dataloader.params, shuffle=False)



In [ ]:
# 获取第一个数据点中的两个字典
dict1, dict2 = train_data[0]

# 合并两个字典
merged_dict = {**dict1, **dict2}
merged_dict

{'camera_feature': tensor([[[[0.3529, 0.3373, 0.3412,  ..., 0.1843, 0.2431, 0.2000],
           [0.3373, 0.3373, 0.3294,  ..., 0.1765, 0.2392, 0.2078],
           [0.3373, 0.3412, 0.3216,  ..., 0.1804, 0.2392, 0.2157],
           ...,
           [0.2235, 0.2275, 0.2196,  ..., 0.1843, 0.1804, 0.1804],
           [0.2157, 0.2275, 0.2196,  ..., 0.1882, 0.1843, 0.1804],
           [0.2275, 0.2235, 0.2314,  ..., 0.1804, 0.1804, 0.1765]],
 
          [[0.3490, 0.3333, 0.3373,  ..., 0.1882, 0.2431, 0.1961],
           [0.3333, 0.3333, 0.3137,  ..., 0.1961, 0.2392, 0.2039],
           [0.3333, 0.3373, 0.3059,  ..., 0.2000, 0.2392, 0.2118],
           ...,
           [0.2431, 0.2471, 0.2392,  ..., 0.2196, 0.2078, 0.2078],
           [0.2353, 0.2471, 0.2392,  ..., 0.2235, 0.2118, 0.2078],
           [0.2471, 0.2431, 0.2510,  ..., 0.2157, 0.2078, 0.2039]],
 
          [[0.3333, 0.3176, 0.3216,  ..., 0.2078, 0.2510, 0.2196],
           [0.3176, 0.3176, 0.3020,  ..., 0.2196, 0.2471, 0.2275],
      

In [ ]:

from diffusion_policy.model.common.normalizer import LinearNormalizer
normalizer=LinearNormalizer()
normalizer.fit(merged_dict)

In [ ]:
merged_dict

{'camera_feature': tensor([[[[0.3529, 0.3373, 0.3412,  ..., 0.1843, 0.2431, 0.2000],
           [0.3373, 0.3373, 0.3294,  ..., 0.1765, 0.2392, 0.2078],
           [0.3373, 0.3412, 0.3216,  ..., 0.1804, 0.2392, 0.2157],
           ...,
           [0.2235, 0.2275, 0.2196,  ..., 0.1843, 0.1804, 0.1804],
           [0.2157, 0.2275, 0.2196,  ..., 0.1882, 0.1843, 0.1804],
           [0.2275, 0.2235, 0.2314,  ..., 0.1804, 0.1804, 0.1765]],
 
          [[0.3490, 0.3333, 0.3373,  ..., 0.1882, 0.2431, 0.1961],
           [0.3333, 0.3333, 0.3137,  ..., 0.1961, 0.2392, 0.2039],
           [0.3333, 0.3373, 0.3059,  ..., 0.2000, 0.2392, 0.2118],
           ...,
           [0.2431, 0.2471, 0.2392,  ..., 0.2196, 0.2078, 0.2078],
           [0.2353, 0.2471, 0.2392,  ..., 0.2235, 0.2118, 0.2078],
           [0.2471, 0.2431, 0.2510,  ..., 0.2157, 0.2078, 0.2039]],
 
          [[0.3333, 0.3176, 0.3216,  ..., 0.2078, 0.2510, 0.2196],
           [0.3176, 0.3176, 0.3020,  ..., 0.2196, 0.2471, 0.2275],
      

In [ ]:
normalizer["trajectory"]

SingleFieldLinearNormalizer(
  (params_dict): ParameterDict(
      (input_stats): Object of type: ParameterDict
      (offset): Parameter containing: [torch.FloatTensor of size 3]
      (scale): Parameter containing: [torch.FloatTensor of size 3]
    (input_stats): ParameterDict(
        (max): Parameter containing: [torch.FloatTensor of size 3]
        (mean): Parameter containing: [torch.FloatTensor of size 3]
        (min): Parameter containing: [torch.FloatTensor of size 3]
        (std): Parameter containing: [torch.FloatTensor of size 3]
    )
  )
)

In [ ]:
traj=normalizer["trajectory"]
stats=traj.get_input_stats()

In [ ]:
traj.params_dict.offset

Parameter containing:
tensor([-1.3234,  1.0198,  1.2312])

In [ ]:
traj.params_dict.scale

Parameter containing:
tensor([0.1753, 0.2238, 2.1160])

In [ ]:
stats.max

Parameter containing:
tensor([13.2534, -0.0883, -0.1092])

In [ ]:
offset=[]
scale=[]
n=0
for data in train_data:
    if n==100:
        break
    dict1, dict2 = data
    merged_dict = {**dict1, **dict2}
    normalizer.fit(merged_dict)
    params_dict=normalizer["trajectory"].params_dict
    #print("offset ",params_dict.offset)
    print("scale ",params_dict.scale)
    offset.append(params_dict.offset)
    scale.append(params_dict.scale)
    n+=1
average_offset=sum(offset)/len(offset)

average_scale=sum(scale)/len(scale)
print(average_offset)
print(average_scale)
    

scale  Parameter containing:
tensor([0.1753, 0.2238, 2.1160])


scale  Parameter containing:
tensor([7.1897e-02, 1.6527e+01, 1.6454e+03])
scale  Parameter containing:
tensor([ 0.0715,  1.5396, 57.4220])
scale  Parameter containing:
tensor([6.4053e-02, 4.7143e+00, 1.0034e+02])
scale  Parameter containing:
tensor([6.0487e-02, 1.5450e+01, 1.6831e+02])
scale  Parameter containing:
tensor([6.5883e-02, 8.3539e+00, 9.1869e+01])
scale  Parameter containing:
tensor([7.4979e-02, 1.4345e+01, 9.6899e+01])
scale  Parameter containing:
tensor([5.5543e-02, 3.6229e+00, 1.2158e+02])
scale  Parameter containing:
tensor([5.2526e-02, 1.2851e+00, 5.3111e+01])
scale  Parameter containing:
tensor([ 0.0699,  2.9899, 58.1336])
scale  Parameter containing:
tensor([ 0.7217, 58.9786, 76.8694])
scale  Parameter containing:
tensor([2.9816e+02, 1.1773e+03, 1.0000e+00])
scale  Parameter containing:
tensor([1.9400e+03, 7.5935e+03, 1.0000e+00])
scale  Parameter containing:
tensor([1.1495e+04, 1.0000e+00, 1.0000e+00])
scale  Parameter containing:
tensor([1.3703e+04, 1.0000e+00, 1.00

In [ ]:
sum(scale)

tensor([199732.1719, 207437.2500, 138446.8438])

In [ ]:
import torch
data=torch.tensor([[ 5.2574e-01, -3.3905e-03, -3.5597e-02],
        [ 6.8287e-01,  5.2506e-03, -5.0342e-02],
        [ 7.3978e-01,  3.0526e-03, -6.9950e-02],
        [ 9.3889e-01, -2.0870e-02, -1.2544e-01],
        [ 1.4480e+00, -1.1697e-01, -2.2646e-01],
        [ 2.2638e+00, -3.5703e-01, -3.7911e-01],
        [ 3.3118e+00, -8.9688e-01, -5.7534e-01],
        [ 4.4899e+00, -1.7859e+00, -7.7512e-01]])


In [ ]:
data = data.reshape(-1,1)

In [ ]:
data

tensor([[ 5.2574e-01],
        [-3.3905e-03],
        [-3.5597e-02],
        [ 6.8287e-01],
        [ 5.2506e-03],
        [-5.0342e-02],
        [ 7.3978e-01],
        [ 3.0526e-03],
        [-6.9950e-02],
        [ 9.3889e-01],
        [-2.0870e-02],
        [-1.2544e-01],
        [ 1.4480e+00],
        [-1.1697e-01],
        [-2.2646e-01],
        [ 2.2638e+00],
        [-3.5703e-01],
        [-3.7911e-01],
        [ 3.3118e+00],
        [-8.9688e-01],
        [-5.7534e-01],
        [ 4.4899e+00],
        [-1.7859e+00],
        [-7.7512e-01]])

In [ ]:
input_min, _ = data.min(axis=0)
input_max, _ = data.max(axis=0)
input_mean = data.mean(axis=0)
input_std = data.std(axis=0)
print("input_min ",input_min)
print("input_max ",input_max)
print("input_mean ",input_mean)
print("input_std ",input_std)

input_min  tensor([-1.7859])
input_max  tensor([4.4899])
input_mean  tensor([0.3746])
input_std  tensor([1.3584])


In [ ]:
output_max=1.
output_min=-1.
range_eps=1e-4

In [ ]:
input_range = input_max - input_min
ignore_dim = input_range < range_eps
input_range[ignore_dim] = output_max - output_min
scale = (output_max - output_min) / input_range
offset = output_min - scale * input_min
offset[ignore_dim] = (output_max + output_min) / 2 - input_min[ignore_dim]

In [ ]:
scale

tensor([0.3187])

In [ ]:
offset

tensor([-0.4309])

In [ ]:
x=data*scale+offset

In [ ]:
x

tensor([[-0.2633],
        [-0.4319],
        [-0.4422],
        [-0.2132],
        [-0.4292],
        [-0.4469],
        [-0.1951],
        [-0.4299],
        [-0.4532],
        [-0.1317],
        [-0.4375],
        [-0.4708],
        [ 0.0306],
        [-0.4681],
        [-0.5030],
        [ 0.2906],
        [-0.5446],
        [-0.5517],
        [ 0.6246],
        [-0.7167],
        [-0.6142],
        [ 1.0000],
        [-1.0000],
        [-0.6779]])

In [1]:
import cv2
import numpy as np

# 创建一个空白图像，尺寸为 500x500 像素，3个颜色通道，背景为黑色
rgb_map = np.zeros((500, 500, 3), dtype=np.uint8)

# 设置圆的参数
center = (250, 250)  # 圆心位置，在图像中心
radius = 50          # 圆的半径
color = (0, 0, 255)  # 红色，OpenCV中颜色顺序是BGR，红色表示 (0, 0, 255)
thickness = -1       # 实心圆

# 画圆
cv2.circle(rgb_map, center, radius, color, thickness)

# 显示图像
cv2.imshow('Circle on Image', rgb_map)
cv2.waitKey(0)  # 等待按键
cv2.destroyAllWindows()  # 关闭所有窗口


: 